<a href="https://www.kaggle.com/code/ramsesmdlc/mnist-classification?scriptVersionId=139573436" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# __0. Libraries__

In [13]:
#Import Libraries 

import numpy as np # linear algebra
import pandas as pd # data processing

import matplotlib as mpl # data visualization
import matplotlib.pyplot as plt # data visualization ("pyplot module", a.k.a. "plt")

    #Each "pyplot" function makes some change to a figure: e.g., creates a figure, creates a...
    #...plotting area in a figure, plots some lines in a plotting area, decorates the plot with...
    #...labels, etc. The various plots we can utilize using Pyplot are Line Plot, Histogram, Scatter,...
    #...3D Plot, Image, Contour, and Polar.

# __1. Important__

Type of machine learning system to build:

1. Supervised Learning: __Classification__
2. Batch Learning (also called "offline learning")
3. Model-based learning

Goal:
1. __It is to take an image of a handwritten single digit, and determine what that digit is.__

2. __Metric__: This competition is evaluated on the categorization accuracy of your predictions (the percentage of images you get correct).

# __2. Loading the Data - Training Dataset__

In [17]:
training_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

## __2.1. General Information__

1. The data file contain __gray-scale images__ of hand-drawn digits, from 0 through 9.

2. Each image is __28 pixels in height and 28 pixels in width, for a total of 784 pixels in total.__

> __Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel__, with higher numbers meaning darker. 

> This pixel-value is an integer between 0 and 255, inclusive.

3. The __training data set, has 785 columns__. The first column, called "label", is the digit that was drawn by the user. The __rest of the columns contain the pixel-values of the associated image__.

4. __Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive__. 

> For example: To locate this pixel ("pixel31") on the image, "pixel31" indicates the pixel is in the fourth column from the left, and the second row from the top.

# __3. Exploratory Data Analysis (EDA) - Training Dataset__

In [19]:
def EDA(training_data):  
  # Print the "shape" of the dataframe
  print("\n" +'\033[1m','\033[94m',"Shape of the dataframe - some metric variables:",'\033[0m', training_data.shape, "\n")
    
  # Print the "head" of the dataframe
  print('\033[1m','\033[94m',"Head of the dataframe - some metric variables:",'\033[0m', "\n")
  print(training_data.head())
  
  # Print the "general information" of the dataframe
  print("\n", '\033[1m','\033[94m',"Information of the dataframe - some metric variables:",'\033[0m', "\n")
  training_data.info()
 
  # Print the "number and percentage" of missing values per column" of some metric variables of the dataframe
  print("\n" +'\033[1m','\033[94m', "Number and percentage of missing values per column of some metric variables of the dataframe:", '\033[0m' + "\n")
  missing = training_data.isnull().sum()
  percent = missing / training_data.shape[0] * 100
  print(pd.concat([missing, percent], axis=1, keys=["Missing", "Percent"]))
    
# Call the function
EDA(training_data)


  Shape of the dataframe - some metric variables:  (42000, 785) 

  Head of the dataframe - some metric variables:  

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0

# __4. Training and the Model - Training Dataset__

## __4.1. Adjusting the Training Dataset__

In [ ]:
#Considerin what is describe 

digitx = training_data[0]
